Find the residuals between ICESat-2 elevation measurements and a reference elevation (taken from IceBridge ATM flyover of Zachariae Isstrom)

Taryn Black, ICESat-2 Hackweek, June 17-21 2019

In [40]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

# Set data location info

In [42]:
home_dir = "/home/jovyan/xtrak/"
cross_file = "data_prod/Intersections_ATM20180418.csv"

# Load ICESat-2 crossover points

ICESat-2 track points that were identified as crossing our OIB ATM reference track in Intersections.ipynb. This file contains both the ICESat-2 elevation data and the OIB ATM data at the crossover points.

In [47]:
cross_df = pd.read_csv(home_dir + cross_file)

cross_df.head()
#cross_df.info()

,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,125293.549537,36.6189,3399.0,35.714657,2019-02-15 10:09:55,1578.0,gt1l
1,125388.182985,35.0684,3402.0,34.106010,2019-02-15 10:09:55,4933.0,gt1r
2,96217.510032,88.3644,2580.0,66.844490,2019-01-16 01:09:49,39471.0,gt3r
3,102715.054408,30.4860,2761.0,29.333950,2019-01-16 01:09:49,19993.0,gt1l
4,102652.754159,33.3447,2759.0,38.831944,2019-01-16 01:09:49,23915.0,gt1r


# Visual check of data

Plot reference elevation profile and crossover data points.

In [67]:
cross_df['t_ATL06'][0]

'2019-02-15 10:09:55'

In [66]:
%matplotlib widget

#c_dict = df['Col_0'].map(pd.Series(data=np.arange(3), index=df['Col_0'].values).to_dict())
#time_colors = cross_df['t_ATL06'].map(pd.Series(data=list('rgb'), index=cross_df['t_ATL06'].values).to_dict())

ax1 = cross_df.plot.scatter(x='dist_along', y='ATM_elev', c='black')
ax2 = cross_df.plot.scatter(x='dist_along', y='z_ATL06', c='t_ATL06', ax=ax1)
plt.xlabel('Distance along track (km)')
plt.ylabel('Elevation (m)')


#plt.colorbar(label='Time')

FigureCanvasNbAgg()

AttributeError: 'PathCollection' object has no property 'column'

# Calculate residual between ICESat-2 track crossovers and reference elevation profile

For each point in the ICESat-2 crossover dataframe `df_IS`, subtract it from the reference elevation at the same distance along-track.